# 3.1 Regression Analysis Interactive Notebook

This notebook provides hands-on implementation of regression techniques for semiconductor manufacturing process engineering. Follow the sections sequentially.

Outline:
1. Import Required Libraries
2. Generate Synthetic Semiconductor Manufacturing Data
3. Exploratory Data Analysis (EDA)
4. Simple Linear Regression
5. Multiple Linear Regression & Feature Engineering
6. Polynomial Regression
7. Regularization (Ridge, Lasso, Elastic Net)
8. Model Validation & Cross-Validation
9. Performance Metrics & Comparison
10. Residual Analysis & Diagnostics
11. Feature Importance & Interpretation
12. SECOM Dataset Analysis
13. Advanced Regression Techniques
14. Production Implementation Pipeline

> Note: Synthetic data replicates common semiconductor relationships (dose vs CD, temperature vs yield, etc.).

## 1. Import Required Libraries

Import core scientific Python libraries for regression analysis and visualization.

Key Libraries:
- numpy / pandas: numerical & tabular processing
- matplotlib / seaborn: visualization
- scikit-learn: modeling & evaluation
- scipy: statistical tests / distributions
- statsmodels: detailed regression diagnostics (optional)

Set a master random seed for reproducibility.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
from pathlib import Path
import math
import json
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', context='notebook')

# Modeling & preprocessing
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor, RANSACRegressor)
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (train_test_split, GridSearchCV, cross_val_score, TimeSeriesSplit, KFold)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

# Stats & diagnostics
from scipy import stats
import statsmodels.api as sm

# Typing
from typing import Tuple, Dict, Any

# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

DATA_DIR = Path('../../../../datasets')
SECOM_DATA_PATH = DATA_DIR / 'secom.data'
SECOM_LABELS_PATH = DATA_DIR / 'secom_labels.data'
SECOM_NAMES_PATH = DATA_DIR / 'secom.names'

print('Library versions:')
print('numpy', np.__version__)
print('pandas', pd.__version__)
import sklearn; print('sklearn', sklearn.__version__)
print('statsmodels', sm.__version__)

# Helper to display section banners
def section(title: str):
    print(f"\n{'='*len(title)}\n{title}\n{'='*len(title)}")

## 2. Generate Synthetic Semiconductor Manufacturing Data

We generate multiple synthetic datasets modeling typical semiconductor process relationships:
- Dose vs Critical Dimension (CD) with focus interaction
- Temperature / Pressure / Gas Flow vs Yield
- Implant Dose / Energy / Anneal parameters vs Threshold Voltage (Vth)

Each dataset embeds controlled noise and known coefficients for validation.

In [ ]:
def generate_photolithography_cd(n=500, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)
    dose = rng.normal(loc=30, scale=3, size=n)
    focus = rng.normal(loc=0, scale=0.15, size=n)
    # True coefficients
    b0, b_dose, b_focus = 100, -1.2, 4.0
    b_dose2, b_focus2, b_inter = 0.02, -6.0, -0.5
    noise = rng.normal(0, 2, size=n)
    cd = (b0 + b_dose*dose + b_focus*focus + b_dose2*dose**2 +
          b_focus2*focus**2 + b_inter*dose*focus + noise)
    return pd.DataFrame({
        'dose': dose,
        'focus': focus,
        'cd': cd
    })

def generate_yield_process(n=600, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed+1)
    temp = rng.normal(450, 15, n)    # CVD temperature (°C)
    pressure = rng.normal(2.5, 0.3, n)  # Torr
    flow = rng.normal(120, 10, n)   # sccm
    time = rng.normal(60, 5, n)     # minutes
    # True relationship (non-linear + interaction)
    noise = rng.normal(0, 3, n)
    yield_pct = (70 + 0.05*(temp-450) - 1.5*(pressure-2.5)**2 + 0.04*flow +
                 0.2*time + 0.0005*(temp-450)*(flow-120) + noise)
    yield_pct = np.clip(yield_pct, 0, 100)
    return pd.DataFrame({
        'temperature': temp,
        'pressure': pressure,
        'flow': flow,
        'time': time,
        'yield_pct': yield_pct
    })

def generate_implant_vth(n=500, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed+2)
    dose = rng.lognormal(mean=13, sigma=0.2, size=n)  # ions/cm^2 scale
    energy = rng.normal(60, 5, size=n)  # keV
    anneal_temp = rng.normal(1000, 20, size=n)  # °C
    anneal_time = rng.normal(30, 3, size=n)  # seconds
    # Arrhenius-like relationship simplified
    k1, k2, k3, k4, k5 = 0.9, -0.002, -0.1, 0.03, -0.00002
    noise = rng.normal(0, 0.02, size=n)
    vth = (k1 + k2*(dose/1e13) + k3*np.log(energy) + k4*np.log(anneal_time) +
           k5*(anneal_temp) + noise)
    return pd.DataFrame({
        'implant_dose': dose,
        'implant_energy': energy,
        'anneal_temp': anneal_temp,
        'anneal_time': anneal_time,
        'vth': vth
    })

cd_df = generate_photolithography_cd()
yield_df = generate_yield_process()
vth_df = generate_implant_vth()

print(cd_df.head())
print(yield_df.head())
print(vth_df.head())

## 3. Exploratory Data Analysis and Visualization

We explore distributions, relationships, and correlations in the synthetic datasets to inform feature engineering decisions.

In [ ]:
# Distribution plots
fig, axes = plt.subplots(1, 3, figsize=(18,4))
axes[0].hist(cd_df['cd'], bins=30, color='steelblue', alpha=0.7)
axes[0].set_title('CD Distribution')
axes[1].hist(yield_df['yield_pct'], bins=30, color='darkgreen', alpha=0.7)
axes[1].set_title('Yield % Distribution')
axes[2].hist(vth_df['vth'], bins=30, color='purple', alpha=0.7)
axes[2].set_title('Vth Distribution')
plt.show()

# Scatter: dose vs cd with color by focus
plt.figure(figsize=(6,4))
plt.scatter(cd_df['dose'], cd_df['cd'], c=cd_df['focus'], cmap='coolwarm', alpha=0.7)
plt.colorbar(label='focus')
plt.xlabel('dose')
plt.ylabel('cd')
plt.title('Dose vs CD (color=focus)')
plt.show()

# Pairplot for yield subset
sns.pairplot(yield_df.sample(200, random_state=RANDOM_SEED), diag_kind='hist')
plt.suptitle('Yield Process Pairplot (Sample)', y=1.02)
plt.show()

# Correlation heatmap for yield process
plt.figure(figsize=(6,4))
cor = yield_df.corr(numeric_only=True)
sns.heatmap(cor, annot=True, cmap='vlag', fmt='.2f')
plt.title('Yield Process Correlations')
plt.show()

cor

## 4. Simple Linear Regression Implementation
We begin with a single predictor relationship (dose → cd) to derive Ordinary Least Squares (OLS) estimates manually and compare with scikit-learn's `LinearRegression`. Steps:
1. Center variables (optional) for numerical stability
2. Compute slope & intercept using closed-form equations
3. Validate against library implementation
4. Plot regression line & residuals

In [ ]:
# Simple Linear Regression: dose -> cd
X = cd_df[['dose']].values
y = cd_df['cd'].values

# Manual OLS (with intercept)
x = X.flatten()
x_mean, y_mean = x.mean(), y.mean()
S_xy = np.sum((x - x_mean)*(y - y_mean))
S_xx = np.sum((x - x_mean)**2)
b1 = S_xy / S_xx
b0 = y_mean - b1 * x_mean
print(f"Manual OLS coefficients: intercept={b0:.4f}, slope={b1:.4f}")

# Sklearn implementation
lr = LinearRegression()
lr.fit(X, y)
print(f"sklearn coefficients: intercept={lr.intercept_:.4f}, slope={lr.coef_[0]:.4f}")

# Predictions & residuals
x_line = np.linspace(x.min(), x.max(), 200)
y_line = b0 + b1 * x_line
y_pred = lr.predict(X)
residuals = y - y_pred

# Plot regression line
plt.figure(figsize=(6,4))
plt.scatter(x, y, alpha=0.5, label='observed')
plt.plot(x_line, y_line, color='red', label='manual OLS line')
plt.xlabel('dose')
plt.ylabel('cd')
plt.title('Simple Linear Regression (Dose → CD)')
plt.legend()
plt.show()

# Residual plot
plt.figure(figsize=(6,3))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted CD')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.show()

print('MAE:', mean_absolute_error(y, y_pred))
print('RMSE:', mean_squared_error(y, y_pred, squared=False))
print('R2:', r2_score(y, y_pred))

## 5. Multiple Linear Regression with Feature Engineering
We extend to multiple predictors (temperature, pressure, flow, time) predicting yield. We'll:
1. Create engineered features (interactions, non-linear terms)
2. Standardize features
3. Fit linear model & evaluate
4. Compare with model sans engineered features

In [ ]:
# Base features
y_base = yield_df['yield_pct'].values
X_base = yield_df[['temperature','pressure','flow','time']].copy()

# Feature Engineering
yield_eng = X_base.copy()
yield_eng['temp_centered'] = yield_eng['temperature'] - yield_eng['temperature'].mean()
yield_eng['pressure_sq'] = yield_eng['pressure']**2
yield_eng['flow_time_inter'] = yield_eng['flow'] * yield_eng['time']
yield_eng['temp_flow_inter'] = yield_eng['temperature'] * yield_eng['flow']

# Train/test split
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_base, y_base, test_size=0.2, random_state=RANDOM_SEED)
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(yield_eng, y_base, test_size=0.2, random_state=RANDOM_SEED)

# Pipelines
base_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler()),
    ('model', LinearRegression())
])
eng_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler()),
    ('model', LinearRegression())
])

base_pipe.fit(X_train_b, y_train_b)
eng_pipe.fit(X_train_e, y_train_e)

pred_b = base_pipe.predict(X_test_b)
pred_e = eng_pipe.predict(X_test_e)

metrics = lambda y,yh: { 'MAE': mean_absolute_error(y,yh), 'RMSE': mean_squared_error(y,yh,squared=False), 'R2': r2_score(y,yh)}
print('Base Metrics:', metrics(y_test_b, pred_b))
print('Engineered Metrics:', metrics(y_test_e, pred_e))

improvement = r2_score(y_test_e, pred_e) - r2_score(y_test_b, pred_b)
print(f"R2 Improvement from feature engineering: {improvement:.4f}")

## 6. Polynomial Regression for Non-Linear Relationships
We model non-linear behavior in CD vs (dose, focus) using polynomial feature expansion. We'll compare degrees and use cross-validation to mitigate overfitting.

In [ ]:
from sklearn.model_selection import KFold

X_poly_base = cd_df[['dose','focus']].values
y_poly = cd_df['cd'].values

results = []
for deg in [1,2,3,4]:
    pipe = Pipeline([
        ('poly', PolynomialFeatures(degree=deg, include_bias=False)),
        ('scale', StandardScaler()),
        ('model', Ridge(alpha=1.0))
    ])
    scores = cross_val_score(pipe, X_poly_base, y_poly, cv=5, scoring='r2')
    results.append({'degree': deg, 'mean_r2': scores.mean(), 'std_r2': scores.std()})

results_df = pd.DataFrame(results)
print(results_df)

best_deg = results_df.sort_values('mean_r2', ascending=False).iloc[0]['degree']
print('Best degree:', best_deg)

best_pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=int(best_deg), include_bias=False)),
    ('scale', StandardScaler()),
    ('model', Ridge(alpha=1.0))
])
best_pipe.fit(X_poly_base, y_poly)

# Visualize fit on a grid for focus=0 slice
x_grid = np.linspace(cd_df['dose'].min(), cd_df['dose'].max(), 200)
focus_zero = np.zeros_like(x_grid)
X_grid = np.vstack([x_grid, focus_zero]).T
cd_pred = best_pipe.predict(X_grid)

plt.figure(figsize=(6,4))
plt.scatter(cd_df['dose'], cd_df['cd'], alpha=0.3, label='observed')
plt.plot(x_grid, cd_pred, color='red', label=f'Polynomial deg={int(best_deg)} (focus=0)')
plt.xlabel('dose')
plt.ylabel('cd')
plt.title('Polynomial Regression Fit (slice focus=0)')
plt.legend()
plt.show()

## 7. Regularization Techniques (Ridge, Lasso, Elastic Net)
We mitigate multicollinearity and overfitting using regularization. We'll tune hyperparameters via grid search and compare performance.

Approach:
- Build pipeline with scaling
- Parameter grids for each model
- 5-fold cross-validation on engineered yield dataset
- Compare R² / MAE / coefficient sparsity

In [ ]:
X_reg = yield_eng.copy()
y_reg = y_base

scale_impute = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler())
])

X_reg_t = scale_impute.fit_transform(X_reg)

ridge_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
lasso_grid = {'alpha': [0.001, 0.01, 0.1, 1.0]}
elastic_grid = {'alpha': [0.01, 0.1, 1.0], 'l1_ratio': [0.2, 0.5, 0.8]}

ridge_cv = GridSearchCV(Ridge(random_state=RANDOM_SEED), ridge_grid, scoring='r2', cv=5)
lasso_cv = GridSearchCV(Lasso(random_state=RANDOM_SEED, max_iter=5000), lasso_grid, scoring='r2', cv=5)
elastic_cv = GridSearchCV(ElasticNet(random_state=RANDOM_SEED, max_iter=5000), elastic_grid, scoring='r2', cv=5)

ridge_cv.fit(X_reg_t, y_reg)
lasso_cv.fit(X_reg_t, y_reg)
elastic_cv.fit(X_reg_t, y_reg)

print('Best Ridge:', ridge_cv.best_params_, 'R2:', ridge_cv.best_score_)
print('Best Lasso:', lasso_cv.best_params_, 'R2:', lasso_cv.best_score_)
print('Best Elastic:', elastic_cv.best_params_, 'R2:', elastic_cv.best_score_)

coef_df = pd.DataFrame({
    'feature': X_reg.columns,
    'ridge_coef': ridge_cv.best_estimator_.coef_,
    'lasso_coef': lasso_cv.best_estimator_.coef_,
    'elastic_coef': elastic_cv.best_estimator_.coef_
})
print(coef_df.head())
print('Non-zero Lasso Coefficients:', (coef_df['lasso_coef']!=0).sum())

## 8. Model Validation and Cross-Validation
We implement multiple validation strategies:
- K-Fold (random shuffle)
- TimeSeriesSplit (simulated chronological data)
- Nested CV for unbiased hyperparameter performance estimate

In [ ]:
# K-Fold CV on polynomial best model from earlier (reusing best_pipe)
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
poly_scores = cross_val_score(best_pipe, X_poly_base, y_poly, cv=kf, scoring='r2')
print('K-Fold R2 mean:', poly_scores.mean(), 'std:', poly_scores.std())

# TimeSeriesSplit simulation (assume yield_df sorted by time surrogate index)
ts = TimeSeriesSplit(n_splits=5)
base_ts_scores = []
for train_idx, test_idx in ts.split(X_reg_t):
    X_tr, X_te = X_reg_t[train_idx], X_reg_t[test_idx]
    y_tr, y_te = y_reg[train_idx], y_reg[test_idx]
    model = Ridge(alpha=ridge_cv.best_params_['alpha'])
    model.fit(X_tr, y_tr)
    y_te_pred = model.predict(X_te)
    base_ts_scores.append(r2_score(y_te, y_te_pred))
print('TimeSeriesSplit R2 scores:', base_ts_scores)
print('TimeSeriesSplit mean R2:', np.mean(base_ts_scores))

# Nested CV (simplified)
outer = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)
outer_scores = []
for train_idx, test_idx in outer.split(X_reg_t):
    X_tr, X_te = X_reg_t[train_idx], X_reg_t[test_idx]
    y_tr, y_te = y_reg[train_idx], y_reg[test_idx]
    inner = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)
    inner_cv = GridSearchCV(Ridge(), {'alpha':[0.1,1,10]}, cv=inner, scoring='r2')
    inner_cv.fit(X_tr, y_tr)
    y_te_pred = inner_cv.best_estimator_.predict(X_te)
    outer_scores.append(r2_score(y_te, y_te_pred))
print('Nested CV R2 scores:', outer_scores, 'mean:', np.mean(outer_scores))

## 9. Performance Metrics and Model Comparison
We compute standard regression metrics plus manufacturing-specific metrics:
- MAE, RMSE, R²
- Prediction Within Spec (PWS)
- Estimated Yield Loss beyond tolerance

In [ ]:
def evaluate_models(models: Dict[str, Any], X, y, tolerance=2.0, spec_low=60, spec_high=100, cost_per_unit=1.0):
    rows = []
    for name, model in models.items():
        y_pred = model.predict(X)
        mae = mean_absolute_error(y, y_pred)
        rmse = mean_squared_error(y, y_pred, squared=False)
        r2 = r2_score(y, y_pred)
        within_spec = ((y_pred >= spec_low) & (y_pred <= spec_high)).mean()
        loss_components = np.maximum(0, np.abs(y - y_pred) - tolerance)
        est_loss = np.sum(loss_components) * cost_per_unit
        rows.append({'model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2,
                     'PWS': within_spec, 'Estimated_Loss': est_loss})
    return pd.DataFrame(rows)

# Prepare models using already fit objects
models_to_compare = {
    'Linear_Base': base_pipe,
    'Linear_Eng': eng_pipe,
    'Ridge': ridge_cv.best_estimator_,
    'Lasso': lasso_cv.best_estimator_,
    'ElasticNet': elastic_cv.best_estimator_
}

metrics_df = evaluate_models(models_to_compare, scale_impute.transform(X_reg), y_reg)
metrics_df.sort_values('R2', ascending=False)

## 10. Residual Analysis and Diagnostics
We assess assumptions:
- Residuals vs Fitted (linearity, homoscedasticity)
- Q-Q Plot (normality)
- Cook's Distance (influence)
- Variance Inflation Factor (VIF) (multicollinearity)
- Durbin-Watson (autocorrelation surrogate)

In [ ]:
# Use engineered linear model residuals
lin_model = eng_pipe.named_steps['model']
X_eng_scaled = eng_pipe.named_steps['scale'].transform(eng_pipe.named_steps['impute'].transform(yield_eng))
lin_pred = lin_model.predict(X_eng_scaled)
lin_res = y_reg - lin_pred

fig, axes = plt.subplots(2,2, figsize=(12,8))
axes[0,0].scatter(lin_pred, lin_res, alpha=0.5)
axes[0,0].axhline(0, color='red', linestyle='--')
axes[0,0].set_title('Residuals vs Fitted')
axes[0,0].set_xlabel('Fitted')
axes[0,0].set_ylabel('Residuals')

stats.probplot(lin_res, dist="norm", plot=axes[0,1])
axes[0,1].set_title('Q-Q Plot')

axes[1,0].hist(lin_res, bins=30, alpha=0.7)
axes[1,0].set_title('Residual Distribution')

axes[1,1].scatter(range(len(lin_res)), lin_res, alpha=0.4)
axes[1,1].axhline(0, color='red', linestyle='--')
axes[1,1].set_title('Residuals vs Index')
plt.tight_layout()
plt.show()

# Statsmodels for influence
X_sm = sm.add_constant(X_eng_scaled)
sm_model = sm.OLS(y_reg, X_sm).fit()
influence = sm_model.get_influence()
(c, p) = influence.cooks_distance

plt.figure(figsize=(6,3))
plt.stem(np.arange(len(c)), c, markerfmt=",", use_line_collection=True)
plt.title("Cook's Distance")
plt.xlabel('Observation')
plt.ylabel('Cook')
plt.show()

# VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_values = []
for i in range(1, X_sm.shape[1]):  # skip constant
    vif_values.append({'feature': yield_eng.columns[i-1], 'VIF': variance_inflation_factor(X_sm, i)})
vif_df = pd.DataFrame(vif_values).sort_values('VIF', ascending=False)
print(vif_df.head())

# Durbin-Watson
from statsmodels.stats.stattools import durbin_watson
dw = durbin_watson(lin_res)
print('Durbin-Watson:', dw)

## 11. Feature Importance and Coefficient Interpretation
We interpret model coefficients using:
- Standardized coefficients
- Confidence intervals (statsmodels)
- Statistical significance (t-stats / p-values)
- Partial dependence style visualization (manual)

In [ ]:
# Standardized coefficients using ridge best (already scaled data used earlier)
ridge_best = ridge_cv.best_estimator_
std_coefs = ridge_best.coef_  # because scale_impute scaled features
importance_df = pd.DataFrame({'feature': X_reg.columns, 'standardized_coef': std_coefs})
importance_df = importance_df.reindex(importance_df.standardized_coef.abs().sort_values(ascending=False).index)
print(importance_df.head())

# Confidence intervals with statsmodels (refit OLS on scaled data)
ols_model = sm.OLS(y_reg, sm.add_constant(X_reg_t)).fit()
conf_int = ols_model.conf_int()
summary_df = pd.DataFrame({
    'feature': ['const'] + list(X_reg.columns),
    'coef': ols_model.params,
    'p_value': ols_model.pvalues,
    'ci_lower': conf_int[0],
    'ci_upper': conf_int[1]
})
print(summary_df.head())

# Plot top absolute standardized coefficients
plt.figure(figsize=(6,5))
imp_top = importance_df.head(10)
plt.barh(imp_top['feature'], imp_top['standardized_coef'])
plt.gca().invert_yaxis()
plt.title('Top Standardized Coefficients (Ridge)')
plt.xlabel('Standardized Coefficient')
plt.show()

# Simple partial dependence approximation for temperature
temp_idx = list(X_reg.columns).index('temperature')
# Vary temperature over quantiles while holding others at mean
quantiles = np.linspace(0.05,0.95,20)
temps = np.quantile(X_reg['temperature'], quantiles)
X_mean = X_reg_t.mean(axis=0)
pd_preds = []
for t in temps:
    X_mod = X_mean.copy()
    # replace scaled value for temp: find original scale then transform
    temp_orig_mean = X_reg['temperature'].mean()
    temp_orig_std = X_reg['temperature'].std()
    X_mod[temp_idx] = (t - temp_orig_mean)/temp_orig_std
    pd_preds.append(ridge_best.predict(X_mod.reshape(1,-1))[0])
plt.figure(figsize=(6,4))
plt.plot(temps, pd_preds, marker='o')
plt.xlabel('Temperature')
plt.ylabel('Predicted Yield')
plt.title('Partial Dependence (Approx) Temperature vs Yield')
plt.show()

## 12. SECOM Dataset Analysis
We load the real SECOM dataset (high-dimensional, missing values). Steps:
1. Load data and labels
2. Handle missing values (remove >50% missing, impute remaining)
3. Scale & dimensionality reduction (PCA)
4. Fit baseline and regularized models
5. Interpret component contributions

In [ ]:
# Load SECOM dataset (unsupervised regression style using continuous proxy target)
# The original target is binary; we will create a synthetic continuous quality metric
# by smoothing label sequence to emulate yield-like continuous response.

if SECOM_DATA_PATH.exists():
    secom_df = pd.read_csv(SECOM_DATA_PATH, sep=' ', header=None)
    labels_df = pd.read_csv(SECOM_LABELS_PATH, sep=' ', header=None, usecols=[0])
    labels = labels_df[0].values
    # Create smoothed continuous target via rolling mean of binary labels (window=25)
    quality_metric = pd.Series(labels).rolling(window=25, min_periods=1, center=True).mean().values

    # Missingness analysis
    missing_pct = secom_df.isna().mean()*100
    keep_cols = missing_pct[missing_pct < 50].index
    secom_reduced = secom_df[keep_cols]

    imputer = SimpleImputer(strategy='median')
    X_secom = imputer.fit_transform(secom_reduced)
    scaler = StandardScaler()
    X_secom_scaled = scaler.fit_transform(X_secom)

    # PCA retain 95% variance
    pca = PCA(n_components=0.95, random_state=RANDOM_SEED)
    X_secom_pca = pca.fit_transform(X_secom_scaled)
    print('Original shape:', secom_df.shape, 'Reduced shape:', X_secom_pca.shape)

    # Train/test split (chronological surrogate)
    n = X_secom_pca.shape[0]
    split = int(n*0.8)
    X_train_s, X_test_s = X_secom_pca[:split], X_secom_pca[split:]
    y_train_s, y_test_s = quality_metric[:split], quality_metric[split:]

    base_lin = LinearRegression().fit(X_train_s, y_train_s)
    ridge_s = Ridge(alpha=1.0).fit(X_train_s, y_train_s)

    preds = {
        'Linear': base_lin.predict(X_test_s),
        'Ridge': ridge_s.predict(X_test_s)
    }

    for name, pr in preds.items():
        print(name, 'MAE', mean_absolute_error(y_test_s, pr), 'RMSE', mean_squared_error(y_test_s, pr, squared=False), 'R2', r2_score(y_test_s, pr))

    # Explained variance per component
    evr = pca.explained_variance_ratio_
    plt.figure(figsize=(6,3))
    plt.plot(np.cumsum(evr)*100)
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Variance (%)')
    plt.title('PCA Cumulative Variance Retained')
    plt.axhline(95, color='red', linestyle='--')
    plt.show()
else:
    print('SECOM dataset files not found; skip Section 12 execution.')

## 13. Advanced Regression Techniques
We explore robust and ensemble methods:
- HuberRegressor (robust to outliers)
- RANSACRegressor (inlier consensus)
- Bagging + Random Forest (non-linear, feature importance)
- Weighted regression (simulate heteroscedastic errors)

In [ ]:
# Robust methods on CD dataset (inject some outliers)
cd_robust = cd_df.copy()
outlier_idx = np.random.choice(cd_robust.index, size=15, replace=False)
cd_robust.loc[outlier_idx, 'cd'] += np.random.normal(40,5,size=15)
X_cd = cd_robust[['dose','focus']].values
y_cd = cd_robust['cd'].values

huber = HuberRegressor().fit(X_cd, y_cd)
ransac = RANSACRegressor(LinearRegression(), random_state=RANDOM_SEED).fit(X_cd, y_cd)
base_lin_cd = LinearRegression().fit(X_cd, y_cd)

for name, model in [('Linear', base_lin_cd), ('Huber', huber), ('RANSAC', ransac)]:
    pred = model.predict(X_cd)
    print(name, 'RMSE', mean_squared_error(y_cd, pred, squared=False))

# Ensemble on yield dataset
rf = RandomForestRegressor(n_estimators=200, max_depth=8, random_state=RANDOM_SEED)
rf.fit(X_reg_t, y_reg)
bag = BaggingRegressor(base_estimator=LinearRegression(), n_estimators=50, random_state=RANDOM_SEED)
bag.fit(X_reg_t, y_reg)
for name, model in [('RF', rf), ('BaggingLin', bag)]:
    pred = model.predict(X_reg_t)
    print(name, 'R2', r2_score(y_reg, pred))

# Weighted regression (simulate higher variance at high flow)
measurement_uncertainty = 1 + 0.01*(yield_eng['flow'] - yield_eng['flow'].mean())**2
weights = 1 / measurement_uncertainty
w_model = LinearRegression()
w_model.fit(X_reg_t, y_reg, sample_weight=weights)
weighted_pred = w_model.predict(X_reg_t)
print('Weighted Linear R2', r2_score(y_reg, weighted_pred))

# Feature importance from RF
rf_imp = pd.Series(rf.feature_importances_, index=X_reg.columns).sort_values(ascending=False).head(10)
plt.figure(figsize=(6,4))
rf_imp.plot(kind='barh')
plt.gca().invert_yaxis()
plt.title('Random Forest Top Feature Importances')
plt.xlabel('Importance')
plt.show()

## 14. Production Implementation Pipeline
We assemble a reusable pipeline function for preprocessing + modeling + evaluation, suitable for deployment adaptation. Includes:
- Data validation
- Preprocessing (impute, scale, select, reduce)
- Model training & persistence
- Monitoring hooks (placeholders)

In [ ]:
import joblib
from datetime import datetime

class RegressionProductionPipeline:
    def __init__(self, model=None, n_components=0.95, k_best=30):
        self.model = model or Ridge(alpha=1.0, random_state=RANDOM_SEED)
        self.n_components = n_components
        self.k_best = k_best
        self.pipeline = None
        self.metadata = {}

    def build(self):
        self.pipeline = Pipeline([
            ('impute', SimpleImputer(strategy='median')),
            ('scale', StandardScaler()),
            ('select', SelectKBest(score_func=f_regression, k=min(self.k_best, X_reg.shape[1]))),
            ('reduce', PCA(n_components=self.n_components, random_state=RANDOM_SEED)),
            ('model', self.model)
        ])
        return self

    def fit(self, X, y):
        if self.pipeline is None:
            self.build()
        self.pipeline.fit(X, y)
        self.metadata = {
            'trained_at': datetime.utcnow().isoformat(),
            'model_type': type(self.model).__name__,
            'n_features_in': X.shape[1],
            'n_components': self.pipeline.named_steps['reduce'].n_components_,
            'k_best': self.pipeline.named_steps['select'].k
        }
        return self

    def predict(self, X):
        return self.pipeline.predict(X)

    def evaluate(self, X, y):
        preds = self.predict(X)
        return {
            'MAE': mean_absolute_error(y, preds),
            'RMSE': mean_squared_error(y, preds, squared=False),
            'R2': r2_score(y, preds)
        }

    def save(self, path: Path):
        joblib.dump({'pipeline': self.pipeline, 'metadata': self.metadata}, path)

    @staticmethod
    def load(path: Path):
        obj = joblib.load(path)
        inst = RegressionProductionPipeline()
        inst.pipeline = obj['pipeline']
        inst.metadata = obj['metadata']
        return inst

# Demonstrate on engineered yield dataset
prod = RegressionProductionPipeline(Ridge(alpha=ridge_cv.best_params_['alpha']))
prod.fit(X_reg_t, y_reg)
metrics_prod = prod.evaluate(X_reg_t, y_reg)
print('Production Pipeline Metrics:', metrics_prod)
print('Metadata:', prod.metadata)

MODEL_PATH = Path('regression_yield_pipeline.joblib')
prod.save(MODEL_PATH)
print('Saved model to', MODEL_PATH)

loaded = RegressionProductionPipeline.load(MODEL_PATH)
print('Loaded metadata:', loaded.metadata)